In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aigl-patentsberta-kaggle-v3/config.json
/kaggle/input/aigl-patentsberta-kaggle-v3/training_args.bin
/kaggle/input/aigl-patentsberta-kaggle-v3/tokenizer.json
/kaggle/input/aigl-patentsberta-kaggle-v3/tokenizer_config.json
/kaggle/input/aigl-patentsberta-kaggle-v3/pytorch_model.bin
/kaggle/input/aigl-patentsberta-kaggle-v3/special_tokens_map.json
/kaggle/input/aigl-patentsberta-kaggle-v3/vocab.txt
/kaggle/input/debertav3small/spm.model
/kaggle/input/debertav3small/config.json
/kaggle/input/debertav3small/README.md
/kaggle/input/debertav3small/tf_model.h5
/kaggle/input/debertav3small/tokenizer_config.json
/kaggle/input/debertav3small/pytorch_model.bin
/kaggle/input/cpc-codes/titles.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


# Credits:
this notebook is based on the notbook ["Getting started with NLP for absolute beginners"](https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners) by @Jeremy Howard

## Imports

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import seaborn as sns
import matplotlib.pyplot as plt

# EDA

In [3]:
train = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
print(train.shape)
train.head()

(36473, 5)


,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [4]:
train.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [5]:
train.groupby('score').count()

,id,anchor,target,context
score,,,,
0.00,7471,7471,7471,7471
0.25,11519,11519,11519,11519
0.50,12300,12300,12300,12300
0.75,4029,4029,4029,4029
1.00,1154,1154,1154,1154


## Score meanings according to [Data Description](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data?select=train.csv)

The scores are in the 0-1 range with increments of 0.25 with the following meanings:

    1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
    0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
    0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
    0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
    0.0 - Unrelated.


# Test Corpus

In [6]:
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")

In [7]:
test.shape

(36, 4)

In [8]:
def cosine(u, v):
    """
    cosine similarity definition
    """
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# External Context

In [9]:
# ajouter le contexte https://www.cooperativepatentclassification.org/Archive
# extraire le contexte du fichier xml

In [10]:
titles = pd.read_csv("../input/cpc-codes/titles.csv")
print(titles.shape)
titles.head()

(260476, 7)


,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


# HuggingFace AutoModelForSequenceClassification

In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from scipy.stats import pearsonr
from scipy.stats import spearmanr # pour trouver les correspondances entre les résultats sur des échelles variées
from sklearn.model_selection import StratifiedGroupKFold

In [12]:
import transformers
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer

import torch
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [13]:
import warnings
import logging
from IPython.display import display, HTML

# Corpora merge

In [14]:
train['section'] = train["context"].str[0]
print(train.shape)
display(train.head())
titles=titles.rename(columns={"code": "context"})
display(titles.head())
train = train.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
train = train.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(train.head())

(36473, 6)


,id,anchor,target,context,score,section
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


,id,anchor,target,context,score,section,context_title,section_title
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES


In [15]:
logging.disable(logging.WARNING)
warnings.simplefilter('ignore')

# PatentSBerta

In [16]:
#model_name = 'AI-Growth-Lab/PatentSBERTa_Kaggle_V3' #(online version)
model_name = '../input/aigl-patentsberta-kaggle-v3'

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
tokenizer.all_special_tokens

['<s>', '</s>', '[UNK]', '<pad>', '<mask>']

In [19]:
def clean(x):
    t = x.lower()
    t = t.replace("[",'')
    t = t.replace(";",'')
    t = t.replace(",",'')
    t = t.replace("]",'')
    t = t.replace(":",'')
    t = t.replace("(",'')
    t = t.replace(")",'')
    t = t.replace("{",'')
    t = t.replace("}",'')
    t = t.replace("/",' ')
    t = t.replace("-",' ')
    return t

In [20]:
train["inputs"] = train["section_title"].apply(clean) + " [SEP] " + train["anchor"] + " [SEP] " + train["target"]

In [21]:
def tok_func(x):
    return tokenizer(x["inputs"])

In [22]:
train_ds = Dataset.from_pandas(train).rename_column('score', 'label')

In [23]:
inps = "anchor","target","context","context_title","section_title"
tok_ds = train_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/37 [00:00<?, ?ba/s]

In [24]:
tok_ds

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [25]:
# bs - batch size
lr,bs = 8e-5,64
# wd: weight decay
wd,epochs = 0.01,4

In [26]:
anchors = train["anchor"].unique()
np.random.shuffle(anchors)

In [27]:
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]

In [28]:
is_val = np.isin(train.anchor, val_anchors)
idxs = np.arange(len(train))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

(9619, 26854)

In [29]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [30]:
def corr(eval_pred): 
    logits, labels = eval_pred
    logits = logits.reshape(-1)
    return {'pearson': np.corrcoef(logits, labels)[0][1]}

In [31]:
#args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    #evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
     #num_train_epochs=epochs, weight_decay=wd, report_to='none')
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine',
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=wd, report_to='none')

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, ignore_mismatched_sizes=True)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr)

In [33]:
dds['train']

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 26854
})

In [34]:
eval = trainer.evaluate()

In [35]:
eval

{'eval_loss': 0.011927393265068531,
 'eval_pearson': 0.9192959048920074,
 'eval_runtime': 8.9017,
 'eval_samples_per_second': 1080.584,
 'eval_steps_per_second': 8.538}

In [36]:
trainer.num_examples(dds['train'])

1718656

In [37]:
#trainer.save_model()

# Test corpus preprocessing

In [38]:
test.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [39]:
test['section'] = test["context"].str[0]
print(test.shape)
display(titles.head())
test = test.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
test = test.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(test.head())

(36, 5)


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


,id,anchor,target,context,section,context_title,section_title
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G,OPTICS,PHYSICS
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G,OPTICS,PHYSICS
2,7aa5908a77a7ec24,el display,illumination,G02,G,OPTICS,PHYSICS
3,442c114ed5c4e3c9,tunneling capacitor,capacitor housing,G11,G,INFORMATION STORAGE,PHYSICS
4,12ca31f018a2e2b9,speed control means,control loop,G05,G,CONTROLLING; REGULATING,PHYSICS


In [40]:
test["inputs"] = test["section_title"].apply(clean) + " [SEP] " + test["anchor"] + " [SEP] " + test["target"]

In [41]:
test_ds = Dataset.from_pandas(test)

In [42]:
test_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'section', 'context_title', 'section_title', 'inputs', '__index_level_0__'],
    num_rows: 36
})

In [43]:
inps = "anchor","target","context","context_title","section_title"
test_tok_ds = test_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [44]:
preds_sberta = trainer.predict(test_tok_ds).predictions.astype(float)
preds_sberta

array([[ 0.48749384],
       [ 0.948901  ],
       [ 0.28694433],
       [ 0.50278115],
       [ 0.31292188],
       [ 0.73349011],
       [ 0.4781965 ],
       [ 0.79074466],
       [ 1.05603766],
       [ 0.2435392 ],
       [ 0.24521858],
       [ 0.55106753],
       [-0.01621631],
       [ 0.5375756 ],
       [ 0.63313073],
       [-0.00538462],
       [ 0.17454851],
       [ 0.79013216],
       [ 0.79604405],
       [-0.00391432],
       [ 0.27687433],
       [ 0.27295172],
       [ 0.52643836],
       [ 0.3617987 ],
       [ 0.01224874],
       [ 0.30376944],
       [ 0.29838696],
       [ 0.53494966],
       [ 0.12322883],
       [ 0.49000978],
       [ 0.48843858],
       [ 0.14339146],
       [-0.01222257],
       [ 0.1442931 ],
       [-0.0065867 ],
       [-0.00724098]])

In [45]:
type(preds_sberta)

numpy.ndarray

In [46]:
del model_name, tokenizer, model

# Deberta-V3_small inference

In [47]:
#model_name = 'microsoft/deberta-v3-small' (online version)
model_name = '../input/debertav3small'

In [48]:
def corr_deberta(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [49]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [50]:
tokenizer.all_special_tokens

['[CLS]', '[SEP]', '[UNK]', '[PAD]', '[MASK]']

In [51]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr_deberta)

In [52]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.053385,0.460184
2,0.062000,0.053394,0.507351
3,0.046100,0.051943,0.519858
4,0.034500,0.054118,0.509917


TrainOutput(global_step=1680, training_loss=0.045755926484153386, metrics={'train_runtime': 293.6093, 'train_samples_per_second': 365.847, 'train_steps_per_second': 5.722, 'total_flos': 695777724027960.0, 'train_loss': 0.045755926484153386, 'epoch': 4.0})

In [53]:
eval_deberta = trainer.evaluate()
eval_deberta

{'eval_loss': 0.05411790311336517,
 'eval_pearson': 0.5099169140365447,
 'eval_runtime': 5.8611,
 'eval_samples_per_second': 1641.162,
 'eval_steps_per_second': 12.967,
 'epoch': 4.0}

In [54]:
preds_deberta = trainer.predict(test_tok_ds).predictions.astype(float)
preds_deberta

array([[ 0.57000232],
       [ 0.85399365],
       [ 0.45876259],
       [ 0.51099473],
       [ 0.25653118],
       [ 0.85706437],
       [ 0.53507996],
       [ 0.68000853],
       [ 0.82697839],
       [ 0.22013141],
       [ 0.2018782 ],
       [ 0.56448948],
       [-0.03417482],
       [ 0.3956126 ],
       [ 0.74591631],
       [ 0.10934766],
       [ 0.2062933 ],
       [ 0.40061393],
       [ 0.83144337],
       [ 0.10363346],
       [ 0.25311753],
       [ 0.36448646],
       [ 0.04307786],
       [ 0.11700297],
       [-0.01482245],
       [ 0.16204326],
       [ 0.22901458],
       [ 0.51212764],
       [ 0.18300085],
       [ 0.45615575],
       [ 0.3860414 ],
       [ 0.24604718],
       [ 0.12833749],
       [ 0.1686345 ],
       [ 0.0669279 ],
       [ 0.14917493]])

In [55]:
del model_name, tokenizer, model

# Ensemble

ensemble methode inspired by Valentin Werner's "In Depth EDA & 3 Model Ensemble" notebook 

In [56]:
test_preds = 0.5 * preds_sberta + (0.5) * preds_deberta

In [57]:
test_preds

array([[ 0.52874808],
       [ 0.90144733],
       [ 0.37285346],
       [ 0.50688794],
       [ 0.28472653],
       [ 0.79527724],
       [ 0.50663823],
       [ 0.7353766 ],
       [ 0.94150802],
       [ 0.23183531],
       [ 0.22354839],
       [ 0.55777851],
       [-0.02519556],
       [ 0.4665941 ],
       [ 0.68952352],
       [ 0.05198152],
       [ 0.1904209 ],
       [ 0.59537305],
       [ 0.81374371],
       [ 0.04985957],
       [ 0.26499593],
       [ 0.31871909],
       [ 0.28475811],
       [ 0.23940084],
       [-0.00128685],
       [ 0.23290635],
       [ 0.26370077],
       [ 0.52353865],
       [ 0.15311484],
       [ 0.47308277],
       [ 0.43723999],
       [ 0.19471932],
       [ 0.05805746],
       [ 0.1564638 ],
       [ 0.0301706 ],
       [ 0.07096698]])

In [58]:
df_submission = pd.DataFrame()

In [59]:
df_submission['id'] = test['id'].copy()

In [60]:
df_submission

,id
0,4112d61851461f60
1,5203a36c501f1b7c
2,7aa5908a77a7ec24
3,442c114ed5c4e3c9
4,12ca31f018a2e2b9
5,9f2279ce667b21dc
6,b9ea2b06a878df6f
7,09e418c93a776564
8,ae0262c02566d2ce
9,03ba802ed4029e4d


In [61]:
preds_list = test_preds.tolist()

In [62]:
preds_list

[[0.5287480801343918],
 [0.9014473259449005],
 [0.37285345792770386],
 [0.5068879425525665],
 [0.2847265303134918],
 [0.7952772378921509],
 [0.50663822889328],
 [0.7353765964508057],
 [0.941508024930954],
 [0.23183530569076538],
 [0.22354838997125626],
 [0.5577785074710846],
 [-0.025195562280714512],
 [0.4665940999984741],
 [0.6895235180854797],
 [0.051981516648083925],
 [0.19042090326547623],
 [0.5953730493783951],
 [0.8137437105178833],
 [0.049859568011015654],
 [0.2649959325790405],
 [0.3187190890312195],
 [0.28475810773670673],
 [0.23940083757042885],
 [-0.0012868549674749374],
 [0.23290634900331497],
 [0.26370076835155487],
 [0.5235386490821838],
 [0.15311483666300774],
 [0.4730827659368515],
 [0.43723998963832855],
 [0.1947193220257759],
 [0.05805745767429471],
 [0.15646380186080933],
 [0.03017060039564967],
 [0.07096697529777884]]

In [63]:
flat_list = [x for xs in preds_list for x in xs]
flat_list

[0.5287480801343918,
 0.9014473259449005,
 0.37285345792770386,
 0.5068879425525665,
 0.2847265303134918,
 0.7952772378921509,
 0.50663822889328,
 0.7353765964508057,
 0.941508024930954,
 0.23183530569076538,
 0.22354838997125626,
 0.5577785074710846,
 -0.025195562280714512,
 0.4665940999984741,
 0.6895235180854797,
 0.051981516648083925,
 0.19042090326547623,
 0.5953730493783951,
 0.8137437105178833,
 0.049859568011015654,
 0.2649959325790405,
 0.3187190890312195,
 0.28475810773670673,
 0.23940083757042885,
 -0.0012868549674749374,
 0.23290634900331497,
 0.26370076835155487,
 0.5235386490821838,
 0.15311483666300774,
 0.4730827659368515,
 0.43723998963832855,
 0.1947193220257759,
 0.05805745767429471,
 0.15646380186080933,
 0.03017060039564967,
 0.07096697529777884]

In [64]:
df_submission['score' ] = flat_list

In [65]:
df_submission

,id,score
0,4112d61851461f60,0.528748
1,5203a36c501f1b7c,0.901447
2,7aa5908a77a7ec24,0.372853
3,442c114ed5c4e3c9,0.506888
4,12ca31f018a2e2b9,0.284727
5,9f2279ce667b21dc,0.795277
6,b9ea2b06a878df6f,0.506638
7,09e418c93a776564,0.735377
8,ae0262c02566d2ce,0.941508
9,03ba802ed4029e4d,0.231835


In [66]:
#submission = datasets.Dataset.from_dict({
  #  'id': test_ds['id'],
 #   'score': preds
#})

df_submission.to_csv('submission.csv', index=False)